In [ ]:
%%writefile cbir.cu

#include <opencv2/opencv.hpp>
#include <iostream>
#include <cuda_runtime.h>

__global__ void calculateImageIntensity(const unsigned char* imageData, size_t width, size_t height, float* intensityHistogram, int numBins, float binSize) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;


    int idx = y * width + x;


    float red = static_cast<float>(imageData[4 * idx]);     // R
    float green = static_cast<float>(imageData[4 * idx + 1]); // G
    float blue = static_cast<float>(imageData[4 * idx + 2]);  // B

    // Convert RGB to intensity
    float intensity = 0.299f * red + 0.587f * green + 0.114f * blue;
    int binIndex = static_cast<int>(floor(intensity / binSize));


    atomicAdd(&intensityHistogram[binIndex], 1);
}




int main(int argc, char** argv) {
    if (argc < 2) {
        std::cerr << "Usage: " << argv[0] << " <Image_Path>" << std::endl;
        return -1;
    }

    std::string imagePath = argv[1];
    cv::Mat image = cv::imread(imagePath, cv::IMREAD_COLOR);
    if (image.empty()) {
        std::cerr << "Error: Image cannot be loaded." << std::endl;
        return -1;
    }

    cv::Mat imageRGBA;
    cv::cvtColor(image, imageRGBA, cv::COLOR_BGR2RGBA);

    const int imgWidth = imageRGBA.cols;
    const int imgHeight = imageRGBA.rows;
    size_t imgSize = imgWidth * imgHeight * 4 * sizeof(unsigned char);


    unsigned char* d_imageData;
    cudaMalloc(&d_imageData, imgSize);


    cudaMemcpy(d_imageData, imageRGBA.data, imgSize, cudaMemcpyHostToDevice);


    const int numBins = 256;
    float binSize = 256.0f / numBins;
    float* d_intensityHistogram;
    size_t histSize = numBins * sizeof(float);
    cudaMalloc(&d_intensityHistogram, histSize);
    cudaMemset(d_intensityHistogram, 0, histSize); // Initialize histogram array to 0


    dim3 blockSize(16, 16);
    dim3 gridSize((imgWidth + blockSize.x - 1) / blockSize.x, (imgHeight + blockSize.y - 1) / blockSize.y);


    calculateImageIntensity<<<gridSize, blockSize>>>(d_imageData, imgWidth, imgHeight, d_intensityHistogram, numBins, binSize);


    float* h_intensityHistogram = new float[numBins];
    cudaMemcpy(h_intensityHistogram, d_intensityHistogram, histSize, cudaMemcpyDeviceToHost);

    delete[] h_intensityHistogram;
    cudaFree(d_imageData);
    cudaFree(d_intensityHistogram);

    return 0;
}




Overwriting cbir.cu


In [ ]:
!nvcc -o imageHistogram cbir.cu


cbir.cu:2:10: fatal error: opencv2/opencv.hpp: No such file or directory
    2 | #include <opencv2/opencv.hpp>
      |          ^~~~~~~~~~~~~~~~~~~~
compilation terminated.


In [ ]:
!nvcc cbir.cu -o cbir
!./cbir
